In [1]:
import sys
sys.path.append('../modules')

In [2]:
### Scenario 1: Authenticated Encryption (Authenticate using SHA-256, then encrypt using RSA) ###
from rsa_utils import generate_keypair, rsa_encrypt, rsa_decrypt
from mac_utils import generate_mac, verify_mac

# === Setup ===
# RSA key pair (K2)
private_key, public_key = generate_keypair(2048)

# Shared MAC key (K1)
mac_key = b'shared_secret_k1'

# Shared delimiter (must not appear in message)
DELIMITER = '||--DELIM--||'

# === Sender Side ===
message = "This is a confidential message."

# 1. Generate MAC
mac = generate_mac(message.encode(), mac_key)

# 2. Append MAC using delimiter
combined = message + DELIMITER + mac.hex()

# 3. Encrypt entire string using RSA
ciphertext = rsa_encrypt(combined, public_key)
print("Encrypted message:", ciphertext)

# === Receiver Side ===
# 4. Decrypt
decrypted_combined = rsa_decrypt(ciphertext, private_key)

# 5. Split using delimiter
try:
    received_msg, received_mac_hex = decrypted_combined.split(DELIMITER)
    received_mac = bytes.fromhex(received_mac_hex)

    # 6. Verify MAC
    is_valid = verify_mac(received_msg.encode(), received_mac, mac_key)

    print("Decrypted Message:", received_msg)
    print("MAC valid?", is_valid)
except ValueError:
    print("Decryption failed: Delimiter not found or corrupted message")


Encrypted message: b'cf\x06\x05\x94\xb5\x05|\x14\x81\x93\xdf\x98\xa0H\x92\xfbG\x89\xe1#_W,\xb6\xe9\x8e\x06\xe9]\xbe\xf1\xd0\x15\xf16\x11\x81\x14\xae+\xee\x1fR\xeaFD;\xd8\xd6\xff"\xee?\xdb\xce\xa3\xc8g\xb9\xf2\xa0X\xc2\xee\x9f\xf9\xa0urL\xc4\xb6~^\xfb\xe3WY\xf7\xe1m\x81\xaf\xf6#\xb8\xff\x16\xfb1\xef\xba\x02\x9e8\xeaor\x93\x8f\xcc\xe8\x7f1=\xbc\xb3;\xd6y\xaf\xf8\xad\xa5\x13\xb3H\xff\x9f\'\xfeD\x9e\x12\xaad&\xbeC\x0b~\x9d\xc4B\xe7\xad\xc0\x97\xfa^\xe4r\xff\xd2\x81\x8e\xe3\xe9\xcb\xfb\xb0e\xfd\xaa;\n>\t\n\x07B\xefx\xb6\xe8s\xe6\xec\xbf\xe5\xab\x0ed\x93\x91\xfb\x9b\x1f\x18\'%\xe7JG\x057\x8c\xeeo\xb5#\xd5\xcd\x137G\xe6\x11\x0c\x99\xbd\x04`O\x05\x1b/Z\xfeV\x11\xb2\xe2\x95hGG3z\xd8\xd5\xf14\x96J\xd3\xe2\xeeiH\x95"H\x87\x958\x0f\xa8\xd4\xca\x96\xb4\xb0<\x00gIE8\xf8\xc6\xa4\x81\xfc:'
Decrypted Message: This is a confidential message.
MAC valid? True


In [3]:
### Scenario 2: DSA Simulation ###
from dsa_utils import generate_dsa_keypair, sign_message, verify_signature 

# Generate DSA Key Pair
private_key, public_key = generate_dsa_keypair()

# Message to Sign 
message = "Verify my identity via digital signature."

# Sender: Sign the message
signature = sign_message(message, private_key)
print("Signature (hex):", signature.hex())

# Receiver: Verify the signature
valid = verify_signature(message, signature, public_key)
print("Signature valid?", valid)

# Tampering check
invalid = verify_signature("Modified message", signature, public_key)
print("Signature valid on tampered message?", invalid)

Signature (hex): 59a33c70e491d9893b3afeccddeed8564a3117d2fb5c6fac72c0aa7d7d95a8a6480cbcea05dbb6e398344f874d208f8aca23edb11cc49abd
Signature valid? True
Signature valid on tampered message? False
